In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  1 18:26:35 2021

@author: rahul
"""
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

import os

os.chdir("D:\\Learning\\Customer Dataloder pytorch youtube\\Machine-Learning-Collection-master\\ML\Pytorch\\Basics\\custom_dataset")




In [4]:
from random import randrange
print(randrange(10))

9


In [5]:
rand = randrange(10)
if rand%2==0:
    print (rand)

0


In [6]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        images = []
        y_labels = []
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)
            
        images.append(image)
        y_labels.append(y_label)

        return images, y_labels




In [237]:
def my_collate(batch):
    
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    data1 = []
    target1 = []
    rand = randrange(4)
    if rand%2==0:
        for i in range(rand):
#             print ("aaaaaaaaaaaaaaaaaaaaaa type of batch", type(batch))
            
#             print ("bbbbbbbbbbbbbbbbbbbbbb batch", batch[1][i])
            
            print ("----------------------inside")
            data1 = [item[0] for item in batch]
            target1 = [item[1] for item in batch]

        if data1:
            rand1 = randrange(len(data1))
            for i in range(rand1):
                data.extend([data1[i]])
                target.extend([target1[i]])
                
    print ("target inside ========", target)
    print ("type of target inside ========", type(target))
    print("length targets inside ========= ", len(target))

    print("length data inside ========= ", len(data))    
    
    target = torch.LongTensor(target)
    return [data, target]

In [238]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# Load Data
dataset = CatsAndDogsDataset(
    csv_file="cats_dogs.csv",
    root_dir="cats_dogs_resized",
    transform=transforms.ToTensor(),
)



In [239]:
# Dataset is actually a lot larger ~25k images, just took out 10 pictures
# to upload to Github. It's enough to understand the structure and scale
# if you got more images.
train_set, test_set = torch.utils.data.random_split(dataset, [5, 5])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, collate_fn=my_collate)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [240]:
# next(iter(train_loader))

In [241]:
import itertools

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (datarah, targetsrah) in enumerate(train_loader):
        # Get data to cuda if possible
        
        print("targetsrah ===================", targetsrah)
        
        data = list(itertools.chain(*datarah))
        targets = list(itertools.chain(*targetsrah))
#         targets = list(itertools.chain(*targets))
#         print("targets = ", targets)
        
        for i in range(len(targets)):
#             print(targets[i])
            targets[i] = targets[i].tolist()
            
        for i in range(len(data)):
#             print(data[i])
            data[i] = data[i].tolist()
        
#         print("targets = ", targets)
#         print (type(targets))
        
        print("length targets = ", len(targets))
        
        print("length data = ", len(data))

        data = torch.tensor(data)
        targets = torch.tensor(targets)
        
        print ("size of the each data in epoches = ", len(data))
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")



target inside ======== [[tensor(0)], [tensor(1)], [tensor(0)], [tensor(0)], [tensor(0)]]
type of target inside ======== <class 'list'>
length targets inside =========  5
length data inside =========  5
targetsrah =================== tensor([[0],
        [1],
        [0],
        [0],
        [0]])
length targets =  5
length data =  5
size of the each data in epoches =  5
Cost at epoch 0 is 7.544194221496582
target inside ======== [[tensor(0)], [tensor(0)], [tensor(1)], [tensor(0)], [tensor(0)]]
type of target inside ======== <class 'list'>
length targets inside =========  5
length data inside =========  5
targetsrah =================== tensor([[0],
        [0],
        [1],
        [0],
        [0]])
length targets =  5
length data =  5
size of the each data in epoches =  5
Cost at epoch 1 is 4.976839065551758
----------------------inside
----------------------inside
target inside ======== [[tensor(1)], [tensor(0)], [tensor(0)], [tensor(0)], [tensor(0)], [tensor(1)], [tensor(0)]]
type 

In [242]:
# t = [[tensor(1)], [tensor(0)], [tensor(0)], [tensor(0)], [tensor(0)], tensor(1)]

In [243]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
           
            x = list(itertools.chain(*x))
            y = list(itertools.chain(*y))

            for i in range(len(x)):
                x[i] = x[i].tolist()
            
            for i in range(len(y)):
                y[i] = y[i].tolist()

            x = torch.tensor(x)
            y = torch.tensor(y)   
            
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()




In [244]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)


Checking accuracy on Training Set
target inside ======== [[tensor(0)], [tensor(0)], [tensor(0)], [tensor(1)], [tensor(0)]]
type of target inside ======== <class 'list'>
length targets inside =========  5
length data inside =========  5
Got 5 / 5 with accuracy 100.00
Checking accuracy on Test Set
Got 3 / 5 with accuracy 60.00
